# Introduction
<hr style = "border:2px solid black" ></hr>


**What?** Pyspark ML



# Import modules
<hr style = "border:2px solid black" ></hr>

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/02 15:22:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()



+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [6]:
#[Age,Experience]----> new feature--->independent feature

In [7]:
featureassembler = VectorAssembler(
    inputCols=["age", "Experience"], outputCol="Independent Features")

In [8]:
output=featureassembler.transform(training)

In [9]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [10]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [11]:
finalized_data=output.select("Independent Features","Salary")

In [12]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [13]:
# train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(
    featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

22/04/02 15:22:38 WARN Instrumentation: [1dfe2541] regParam is zero, which might cause numerical instability and overfitting.
22/04/02 15:22:38 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/02 15:22:38 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/02 15:22:38 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
### Coefficients
regressor.coefficients

DenseVector([-0.0, 1666.6667])

In [15]:
### Intercepts
regressor.intercept

13333.333333333807

In [16]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [17]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|16666.666666666693|
|          [24.0,3.0]| 20000| 18333.33333333336|
|          [30.0,8.0]| 25000|26666.666666666664|
+--------------------+------+------------------+



/opt/anaconda3/envs/trainingAI/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [18]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1555.5555555555366, 2444444.4444443868)

# References
<hr style = "border:2px solid black" ></hr>


- https://github.com/krishnaik06/Pyspark-With-Python
- https://www.youtube.com/watch?v=l6dx_0LobsA
    
